In [2]:
#import packages
import modal
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from PIL import Image
# import torch
# from torch.utils.data import Dataset, DataLoader
# import torchvision.transforms as transforms
# import torch.nn as nn
# import segmentation_models_pytorch as smp
# from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# Set up Modal App for intreeligent
app = modal.App("intreeligent-v1")

In [4]:
# Set up Modal image
image = (
    modal.Image.debian_slim(python_version="3.13.7")
    .pip_install_from_requirements("requirements.txt")
    .apt_install("libgl1-mesa-glx" "libglib2.0-0")  # for cv2
)

In [5]:
# Create volume for data
volume = modal.Volume.from_name("treesdataset")


In [6]:
# Converting XML annotations to masks
def xml_to_mask(xml_path, img_shape):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    mask = np.zeros(img_shape[:2], dtype=np.uint8)

    for obj in root.findall('.//object'):
        tree_element = obj.find('tree')
        if tree_element is not None:
            bbox = obj.find('bndbox')
            if bbox is not None:
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)
                
                center_x = (xmin + xmax) // 2
                center_y = (ymin + ymax) // 2
                width = (xmax - xmin) // 2
                height = (ymax - ymin) // 2

                cv2.ellipse(mask, (center_x, center_y), (width, height), 0, 0, 360, 1, -1)

    return mask

In [ ]:
class TreeDataset(Dataset):
    def __init__(self, image_dir, xml_dir, transform=None):
        self.image_dir = image_dir
        